# Data Acquisition: Scraping

### Imports

In [ ]:
# Standard data manipulation libraries
import pandas as pd
import numpy as np

# Webscraping imports
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time

In [ ]:
data = pd.read_csv('./data/data.csv')

In [ ]:
# convert ssvid column to list of numbers
ssvids = data['ssvid'].values.tolist()
len(ssvids)

In [ ]:
# Split into 20 lists of 25,000 numbers to feed scraping tool
groups = [ssvids[x:x+25000] for x in range(0, len(ssvids), 25000)]

## Algorithm 1

In [ ]:
imo_reg = []
imo_notreg = []

In [ ]:
# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicarose")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("Fishing0")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()


# Create a function that searches for each MMSI number in the IMO database
# and classifies them into registered/not registered lists 

def check_registration(lst, group_num):
    for num in lst:
        
        try:
            driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
            driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
            driver.implicitly_wait(5)
            table = driver.find_element_by_xpath('//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]')
            #table = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]')))
            rows = table.find_elements(By.TAG_NAME, "tr")
            if len(rows) <= 1:
                imo_notreg.append(num)
            elif len(rows) >= 2:
                imo_reg.append(num)
            driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()
            pd.Series(imo_notreg).to_csv(f'./imo_notreg{group_num}.csv')
            pd.Series(imo_reg).to_csv(f'./imo_reg{group_num}.csv')
        
        except NoSuchElementException:
            print(num)
            
check_registration(groups[13], 13)

In [ ]:
# Check list length (to confirm scrape running properly)
print(f'Registered:{len(imo_reg)} Unregistered:{len(imo_notreg)}')

In [ ]:
# check for duplicates
len(imo_reg) != len(set(imo_reg))

In [ ]:
# read in each list of categorized numbers (create function to do this?)
nr1 = pd.read_csv('./imo_notreg.csv')
r1 = pd.read_csv('./imo_reg.csv')
nr2 = pd.read_csv('./imo_notreg2.csv')
r2 = pd.read_csv('./imo_reg2.csv')
nr3 = pd.read_csv('./imo_notreg3.csv')
r3 = pd.read_csv('./imo_reg3.csv')
nr4 = pd.read_csv('./imo_notreg4.csv')
r4 = pd.read_csv('./imo_reg4.csv')
nr5 = pd.read_csv('./imo_notreg5.csv')
r5 = pd.read_csv('./imo_reg5.csv')
nr6 = pd.read_csv('./imo_notreg6.csv')
r6 = pd.read_csv('./imo_reg6.csv')
nr7 = pd.read_csv('./imo_notreg7.csv')
r7 = pd.read_csv('./imo_reg7.csv')
nr8 = pd.read_csv('./imo_notreg8.csv')
r8 = pd.read_csv('./imo_reg8.csv')
nr9 = pd.read_csv('./imo_notreg9.csv')
r9 = pd.read_csv('./imo_reg9.csv')
nr10 = pd.read_csv('./imo_notreg10.csv')
r10 = pd.read_csv('./imo_reg10.csv')
nr11 = pd.read_csv('./imo_notreg11.csv')
r11 = pd.read_csv('./imo_reg11.csv')
nr12 = pd.read_csv('./imo_notreg12.csv')
r12 = pd.read_csv('./imo_reg12.csv')
nr13 = pd.read_csv('./imo_notreg13.csv')
r13 = pd.read_csv('./imo_reg13.csv')

In [ ]:
# concatenate the registered and non-registered IMO lists 

imo_reg = pd.concat([r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13], axis=0).reset_index(drop=True)

imo_notreg = pd.concat([nr1,nr2,nr3,nr4,nr5,nr6,nr7,nr8,nr9,nr10,nr11,nr12,nr13], axis=0).reset_index(drop=True)

print(f'Registered:{len(imo_reg)} Unregistered:{len(imo_notreg)}')

In [ ]:
# Drop the unnamed column
imo_reg.drop(columns=['Unnamed: 0'], inplace=True)
imo_notreg.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
# Save files as csv
imo_reg.to_csv('./data/imo_reg.csv')
imo_notreg.to_csv('./data/imo_notreg.csv')

In [ ]:
imo_reg.head()

In [ ]:
imo_notreg.head()

In [ ]:
# import data
imo_reg_nums = pd.read_csv("./data/imo_reg.csv")

In [ ]:
# create a list of numbers to feed second algorithm
imo_reg_nums = imo_reg['0'].values.tolist()
len(imo_reg_nums)

## Algorithm 2

- Added second if statement to catch multiple rows.
- The header of the table is also being counted as a row - increased len of if statements. One search result will have len of 2.
- used code from previous work where successful on scraping one search result
- Only remaining issue is skipping the first row of multiple search results

In [ ]:
# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicaertel")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("Fishing123")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()

# Create lists to hold scraped data
name, imo_num, flag, call_sign, mmsi, vessel_type, built, tonnage, owner = [],[],[],[],[],[],[],[],[]

count = 0
length = len(imo_reg_nums)

# For each number in ssvids
for num in imo_reg_nums:
    count += 1
    if count % 50 == 0:
        print(f'{count} have been scraped out of {length}', end = '\r')
        
    # Enter number into MMSI field and click search
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
    driver.implicitly_wait(5)
    
    # define the results table and rows
    table = driver.find_element_by_id('ctl00_bodyPlaceHolder_Default_gridShips')
    rows = table.find_elements(By.TAG_NAME, "tr")
    
    # if there is only one search result
    if len(rows) == 2:
        
        # Find and click the result
        driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr').click()
#         WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH,f'/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[{row}]')))
#         driver.find_element_by_xpath(f'//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/tr[{row}]').click()
#         driver.find_element_by_xpath('//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/td').click()
        
        # scrape and append the data into lists
        name.append(driver.find_element_by_id("sNameCur").text)
        imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]").text)
        flag.append(driver.find_element_by_id("sFlagCur").text)
        call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]").text)
        mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]").text)
        vessel_type.append(driver.find_element_by_id("sTypeCur").text)
        built.append(driver.find_element_by_id("sBuildDateCur").text)
        tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]").text)
        owner.append(driver.find_element_by_id("sRegOwnerCur").text)
        time.sleep(1)
        
        # return to the search results page and clear search field
        driver.execute_script("window.history.go(-1)")
        
        # Clear search field    
        driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()
        
    # if there are multiple search results, loop through each result
    elif len(rows) > 2:
        for row in range(1, len(rows)+1):

            # wait until element appears, then click it
            WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH,f'/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[{row}]')))
            driver.find_element_by_xpath(f'//*[@id="ctl00_bodyPlaceHolder_Default_gridShips"]/tbody/tr[{row}]').click()

            # scrape and append the data into lists
            name.append(driver.find_element_by_id("sNameCur").text)
            imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]").text)
            flag.append(driver.find_element_by_id("sFlagCur").text)
            call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]").text)
            mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]").text)
            vessel_type.append(driver.find_element_by_id("sTypeCur").text)
            built.append(driver.find_element_by_id("sBuildDateCur").text)
            tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]").text)
            owner.append(driver.find_element_by_id("sRegOwnerCur").text)
            time.sleep(1)

            # return to the search results page to scrape next result
            driver.execute_script("window.history.go(-1)")

    # Clear search field
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()

In [ ]:
name, imo_num, flag, call_sign, mmsi, vessel_type, built, tonnage, owner

In [ ]:
# transform lists into pandas dataframe
imo = pd.DataFrame({'name': name,
                     'imo_num': imo_num,
                     'flag': flag,
                     'call_sign': call_sign,
                     'mmsi': mmsi,
                     'vessel_type': vessel_type,
                     'built' : built,
                     'tonnage': tonnage,
                     'owner': owner})

In [ ]:
imo.shape

In [ ]:
imo.to_csv('imo_scrape.csv', index=False)

## References
- https://stackoverflow.com/questions/60362018/macos-catalinav-10-15-3-error-chromedriver-cannot-be-opened-because-the-de
- https://selenium-python.readthedocs.io/locating-elements.html
- https://stackoverflow.com/questions/27338742/how-do-i-send-a-delete-keystroke-to-a-text-field-using-selenium-with-python
- https://stackoverflow.com/questions/12057118/iterate-through-table-in-selenium-2-webdriver-python
- https://stackoverflow.com/questions/9671224/split-a-python-list-into-other-sublists-i-e-smaller-lists